In [ ]:
#@title
# Upload: 
# gan_faces_32.zip

# 8.4. Auto-Encoders for Reconstruction

In [ ]:
#@title 8.4.1. Import some necessary packages
import numpy as np
import pandas as pd
import glob

from sklearn.model_selection import train_test_split

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
#@title 8.4.2. Data processing
# https://thispersondoesnotexist.com/ 
!unzip gan_faces_32.zip
batch_files = glob.glob('/content/*.npy')

datain = np.empty((len(batch_files) * 200,32,32,1))
c = 0
for i0 in range(len(batch_files)):
  datain[c:c+200] = np.load(batch_files[i0])
  c             = c + 200

datain = datain/255

# devide data into training and testing 
datain_tr, datain_te, _, _ = train_test_split(datain, datain, test_size = 0.1, random_state = 42) # we just define dataou to be datain just to run this line without any issue


# input information for CNN layers
shape0_tr = datain_tr.shape[0]
shape0_te = datain_te.shape[0]
shape1    = int(datain_tr.shape[1])
shape2    = int(datain_tr.shape[2])
shape3    = 1

print("#####################################")
print("size of all      data: {}".format(datain.shape   ) )
print("size of training data: {}".format(datain_tr.shape) )
print("size of testing  data: {}".format(datain_te.shape) )
print("#####################################")



In [ ]:
#@title 8.4.3. Data observation
num_data = 5
f, ax           = plt.subplots(num_data, num_data, sharex= True, sharey= True, figsize=(num_data*3,num_data*3))
f.subplots_adjust(hspace=0)

c = 0
for i0 in range(num_data):
  for i1 in range(num_data):
    ax[i0,i1].imshow(datain[c,:,:,0], cmap='gray' )
    c = c + 1

plt.axis('off')


In [ ]:
#@title 8.4.4. Encoder model
model_encoder = models.Sequential()

model_encoder.add(layers.Conv2D(filters = 16, kernel_size=(3, 3), strides=(1,1), input_shape=(shape1, shape2, shape3)))
model_encoder.add(layers.LayerNormalization())
model_encoder.add(layers.ReLU())
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model_encoder.add(layers.Conv2D(filters = 32, kernel_size=(3, 3), strides=(1, 1)))
model_encoder.add(layers.LayerNormalization())
model_encoder.add(layers.ReLU())
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model_encoder.add(layers.Conv2D(filters = 64, kernel_size=(3, 3), strides=(1, 1)))
model_encoder.add(layers.LayerNormalization())
model_encoder.add(layers.ReLU())
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.Flatten())
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.Dense(768, activation='relu'))
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.Dense(386, activation='relu'))
model_encoder.add(layers.Dropout(rate = 0.1))

# Common layer between the encoder and the decoder

model_encoder.add(layers.Dense(128, activation='relu'))
model_encoder.add(layers.Dropout(rate = 0.1))

#model.compile(optimizer='adam', loss = "mean_squared_error")

model_encoder.summary()

In [ ]:
#@title 8.4.5. Decoder model
model_decoder = models.Sequential()

model_decoder.add(layers.Dense(128, activation='relu',  input_dim = 128))
model_decoder.add(layers.Dropout(rate = 0.1))

model_decoder.add(layers.Dense(386, activation='relu'))
model_decoder.add(layers.Dropout(rate = 0.1))

model_decoder.add(layers.Dense(786, activation='relu'))
model_decoder.add(layers.Dropout(rate = 0.1))

model_decoder.add(layers.Dense(1024, activation='relu'))
model_decoder.add(layers.Dropout(rate = 0.1))

model_decoder.add(layers.Reshape((4, 4, 64)))
model_decoder.add(layers.LayerNormalization())
model_decoder.add(layers.Conv2DTranspose(filters = 16, kernel_size=(3, 3), strides=(1, 1)) )

model_decoder.add(layers.Dropout(rate = 0.1))
model_decoder.add(layers.UpSampling2D((2, 2)) )
model_decoder.add(layers.LayerNormalization())
model_decoder.add(layers.Conv2DTranspose(filters = 32, kernel_size=(3,3), strides=(1, 1)) )

model_decoder.add(layers.Dropout(rate = 0.1))
model_decoder.add(layers.UpSampling2D((2, 2)) )
model_decoder.add(layers.LayerNormalization())
model_decoder.add(layers.Conv2DTranspose(filters = 1, kernel_size=(5,5), strides=(1, 1)) )

model_decoder.summary()

In [ ]:
#@title 8.4.6. Auto-Encoder model
model_autoencoder = models.Sequential()
model_autoencoder.add(model_encoder)
model_autoencoder.add(model_decoder)

model_autoencoder.compile(optimizer='adam', loss = "mean_squared_error")

model_autoencoder.summary()


In [ ]:
#@title 8.4.7. Training the Auto-Encoder model
history = model_autoencoder.fit(datain_tr, datain_tr, epochs = 500, batch_size = 128, verbose = 1, shuffle=True, validation_split=0.1)

In [ ]:
#@title 8.4.8. Training and testing estimations
dataes_tr = model_autoencoder.predict(datain_tr)
dataes_te = model_autoencoder.predict(datain_te)

In [ ]:
#@title 8.4.9. Observe some training estimations
f, ax = plt.subplots(10, 2, sharex= True, sharey= True, figsize=(5,15))
f.subplots_adjust(hspace=0)
for i0 in range(10):
  ax[i0,0].imshow(datain_tr[i0,:,:,0], cmap='gray')
  ax[i0,1].imshow(dataes_tr[i0,:,:,0], cmap='gray')

In [ ]:
#@title 8.4.10. Observe some testing estimations
f, ax = plt.subplots(10, 2, sharex= True, sharey= True, figsize=(5,15))
f.subplots_adjust(hspace=0)
for i0 in range(10):
  ax[i0,0].imshow(datain_te[i0,:,:,0], cmap='gray')
  ax[i0,1].imshow(dataes_te[i0,:,:,0], cmap='gray')

In [ ]:
#@title 8.4.11. Training and validation plots 
plt.figure(figsize=[8,8])
plt.plot(history.history['loss'],'--')
plt.plot(history.history['val_loss'],'-')
plt.title('model loss', fontsize = 20)
plt.ylabel('loss', fontsize = 20)
plt.xlabel('epoch', fontsize = 20)
plt.legend(['training loss','validation loss'], fontsize = 20)
plt.show()

# 8.5. Auto-Encoders for Denoising

In [ ]:
#@title 8.5.1. Import some necessary packages
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
#@title 8.5.2. Data processing
# https://thispersondoesnotexist.com/ 
#!unzip gan_faces_32.zip
batch_files = glob.glob('/content/*.npy')

datain = np.empty((len(batch_files) * 200,32,32,1))
c = 0
for i0 in range(len(batch_files)):
  datain[c:c+200] = np.load(batch_files[i0])
  c             = c + 200

datain = datain/255

# devide data into training and testing 
datain_tr, datain_te, _, _ = train_test_split(datain, datain, test_size = 0.1, random_state = 42) # we just define dataou to be datain just to run this line without any issue


# reshape for CNN layers
shape0_tr = datain_tr.shape[0]
shape0_te = datain_te.shape[0]
shape1    = int(datain_tr.shape[1])
shape2    = int(datain_tr.shape[2])
shape3    = 1

# Synthetic noisy distribution: apply a gaussian noise matrix and clip the images between 0 and 1. (ref: https://blog.keras.io/building-autoencoders-in-keras.html)
noise_factor    = 0.05
datain_tr_noisy = datain_tr + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=datain_tr.shape) 
datain_te_noisy = datain_te + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=datain_te.shape) 

datain_tr_noisy   = np.clip(datain_tr_noisy, 0., 1.)
datain_te_noisy   = np.clip(datain_te_noisy, 0., 1.)

datain_noisy      = np.concatenate((datain_tr_noisy, datain_te_noisy), axis = 0)
datain_original   = np.concatenate((datain_tr      , datain_te      ), axis = 0)

print("#####################################")
print("size of all      data: {}".format(datain.shape   ) )
print("size of training data: {}".format(datain_tr.shape) )
print("size of testing  data: {}".format(datain_te.shape) )
print("#####################################")


In [ ]:
#@title 8.5.3. Noisy data observation
# Original samples
num_data = 5
f, ax           = plt.subplots(num_data, num_data, sharex= True, sharey= True, figsize=(num_data*3,num_data*3))
f.subplots_adjust(hspace=0)
f.suptitle('Original', fontsize = 20)

c = 0
for i0 in range(num_data):
  for i1 in range(num_data):
    ax[i0,i1].imshow(datain_original[c,:,:,0], cmap='gray' )
    c = c + 1

plt.axis('off')

# Noisy samples 
num_data = 5
f, ax           = plt.subplots(num_data, num_data, sharex= True, sharey= True, figsize=(num_data*3,num_data*3))
f.subplots_adjust(hspace=0)
f.suptitle('Noisy', fontsize = 20)

c = 0
for i0 in range(num_data):
  for i1 in range(num_data):
    ax[i0,i1].imshow(datain_noisy[c,:,:,0], cmap='gray' )
    c = c + 1

plt.axis('off')

In [ ]:
#@title 8.5.4. Encoder model
model_encoder = models.Sequential()

model_encoder.add(layers.Conv2D(filters = 16, kernel_size=(3, 3), strides=(1,1), input_shape=(shape1, shape2, shape3)))
model_encoder.add(layers.LayerNormalization())
model_encoder.add(layers.ReLU())
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model_encoder.add(layers.Conv2D(filters = 32, kernel_size=(3, 3), strides=(1, 1)))
model_encoder.add(layers.LayerNormalization())
model_encoder.add(layers.ReLU())
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model_encoder.add(layers.Conv2D(filters = 64, kernel_size=(3, 3), strides=(1, 1)))
model_encoder.add(layers.LayerNormalization())
model_encoder.add(layers.ReLU())
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.Flatten())
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.Dense(768, activation='relu'))
model_encoder.add(layers.Dropout(rate = 0.1))

model_encoder.add(layers.Dense(386, activation='relu'))
model_encoder.add(layers.Dropout(rate = 0.1))

# Common layer between the encoder and the decoder

model_encoder.add(layers.Dense(128, activation='relu'))
model_encoder.add(layers.Dropout(rate = 0.1))

#model.compile(optimizer='adam', loss = "mean_squared_error")

model_encoder.summary()

In [ ]:
#@title 8.5.5. Decoder model
model_decoder = models.Sequential()

model_decoder.add(layers.Dense(128, activation='relu',  input_dim = 128))
model_decoder.add(layers.Dropout(rate = 0.1))

model_decoder.add(layers.Dense(386, activation='relu'))
model_decoder.add(layers.Dropout(rate = 0.1))

model_decoder.add(layers.Dense(786, activation='relu'))
model_decoder.add(layers.Dropout(rate = 0.1))

model_decoder.add(layers.Dense(1024, activation='relu'))
model_decoder.add(layers.Dropout(rate = 0.1))

model_decoder.add(layers.Reshape((4, 4, 64)))
model_decoder.add(layers.LayerNormalization())
model_decoder.add(layers.Conv2DTranspose(filters = 16, kernel_size=(3, 3), strides=(1, 1)) )

model_encoder.add(layers.Dropout(rate = 0.1))
model_decoder.add(layers.UpSampling2D((2, 2)) )
model_decoder.add(layers.LayerNormalization())
model_decoder.add(layers.Conv2DTranspose(filters = 32, kernel_size=(3,3), strides=(1, 1)) )

model_encoder.add(layers.Dropout(rate = 0.1))
model_decoder.add(layers.UpSampling2D((2, 2)) )
model_decoder.add(layers.LayerNormalization())
model_decoder.add(layers.Conv2DTranspose(filters = 1, kernel_size=(5,5), strides=(1, 1)) )

model_decoder.summary()

In [ ]:
#@title 8.5.6. Auto-Encoder model
model_autoencoder = models.Sequential()
model_autoencoder.add(model_encoder)
model_autoencoder.add(model_decoder)

model_autoencoder.compile(optimizer='adam', loss = "mean_squared_error")

model_autoencoder.summary()

In [ ]:
#@title 8.5.7. Training the Auto-Encoder model
history = model_autoencoder.fit(datain_tr, datain_tr_noisy, epochs = 1000, batch_size = 256, verbose = 1, shuffle=True, validation_split=0.1)

In [ ]:
#@title 8.5.8. Training and testing estimations
dataes_tr = model_autoencoder.predict(datain_tr)
dataes_te = model_autoencoder.predict(datain_te)

In [ ]:
#@title 8.5.9. Observe some training estimations
f, ax = plt.subplots(10, 2, sharex= True, sharey= True, figsize=(5,15))
f.subplots_adjust(hspace=0)
for i0 in range(10):
  ax[i0,0].imshow(datain_tr_noisy[i0,:,:,0], cmap='gray')
  ax[i0,1].imshow(dataes_tr[i0,:,:,0], cmap='gray')

In [ ]:
#@title 8.5.10. Observe some testing estimations
f, ax = plt.subplots(10, 2, sharex= True, sharey= True, figsize=(2,15))
f.subplots_adjust(hspace=0)
for i0 in range(10):
  ax[i0,0].imshow(datain_te_noisy[i0,:,:,0], cmap='gray')
  ax[i0,1].imshow(dataes_te[i0,:,:,0], cmap='gray')

In [ ]:
#@title 8.5.11. Training and validation plots 
plt.figure(figsize=[8,8])
plt.plot(history.history['loss'],'--')
plt.plot(history.history['val_loss'],'-')
plt.title('model loss', fontsize = 20)
plt.ylabel('loss', fontsize = 20)
plt.xlabel('epoch', fontsize = 20)
plt.legend(['training loss','validation loss'], fontsize = 20)
plt.show()

# 8.6. Restricted Boltzmann Machine (RBM)

In [ ]:
#@title 8.6.1. Import some necessary packages
import numpy as np
import pandas as pd
import glob

from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as fun_mse

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models

# **Reminder**


---


# One iteration in RBM


$\boldsymbol{H}=\operatorname{sigmoid}(\boldsymbol{X} \boldsymbol{W}+\boldsymbol{O B})$

$\widetilde{\boldsymbol{H}}=\boldsymbol{H} \leq \boldsymbol{R}$

$\overline{\boldsymbol{X}}=\operatorname{sigmoid}\left(\widetilde{\boldsymbol{H}} \boldsymbol{W}^{T}+\boldsymbol{O} \boldsymbol{C}\right)$

$\overline{\boldsymbol{H}}=\operatorname{sigmoid}(\overline{\boldsymbol{X}} \boldsymbol{W}+\boldsymbol{O} \boldsymbol{B})$


---

# Learning (Constructive Divergence - CD -)


$\boldsymbol{W}^{s+1}=\boldsymbol{W}^{S}+\Delta \boldsymbol{W}^{S}$

$\boldsymbol{B}^{s+1}=\boldsymbol{B}^{S}+\Delta \boldsymbol{B}^{s}$

$\boldsymbol{C}^{S+1}=\boldsymbol{C}^{S}+\Delta \boldsymbol{C}^{S}$

$\Delta \boldsymbol{W}^{s}=\mu \Delta \boldsymbol{W}^{s-1}+\eta \frac{\left(\boldsymbol{X}^{T} \boldsymbol{H}-\overline{\boldsymbol{X}}^{T} \overline{\boldsymbol{H}}\right)}{N}-W_{\text {cost}} \times \boldsymbol{W}^{s}$

$\Delta \boldsymbol{B}^{s}=\mu \Delta \boldsymbol{B}^{s-1}+\eta\left(\sum_{n=1}^{N} \boldsymbol{H}_{n}-\sum_{n=1}^{N} \overline{\boldsymbol{H}}_{n}\right)$

$\Delta \boldsymbol{\Psi}^{s}=\mu \Delta \boldsymbol{\Psi}^{s-1}+\eta\left(\sum_{n=1}^{N} \boldsymbol{X}_{n}-\sum_{n=1}^{N} \overline{\boldsymbol{X}}_{n}\right)$



In [ ]:
#@title 8.6.2. Define RBM function

#num_hidden = 128 
#num_var    = 1024

# To generate initial parameters and variables (weights, biases, etc.)
def fun_rbm_options(num_hidden, num_var, random_state = 42):
  np.random.seed(random_state)
  options                         = {}

  options['weights']              = np.random.rand(num_var, num_hidden) * 0.1
  options['delta_weights']        = np.zeros((num_var, num_hidden))
  options['weight_cost']          = 0.0002

  options['hidden_biases' ]       = np.zeros((1, num_hidden))
  options['delta_hidden_biases']  = np.zeros((1, num_hidden))

  options['visible_biases']       = np.zeros((1,num_var))
  options['delta_visible_biases'] = np.zeros((1,num_var))

  options['mu']                   = 0.9 # recom: 0.9 for the a number of early epochs and 0.5 for the rest
  options['learning_rate' ]       = 0.01

  return options

# The RBM function
def fun_rbm(data, options):
  # Define sigmoid function
  def fun_sigmoid(x): return 1/(1+np.exp(-x))

  # size of batch of visible inputs 
  batch_size = data.shape[0]
  num_var    = data.shape[1]
  num_hidden = options['weights'].shape[1]

  # One iteration
  H       = fun_sigmoid( np.dot(data    , options['weights'])             + options['hidden_biases' ].repeat(batch_size, axis = 0) ) 
  H_tilda = ( H < np.random.rand(batch_size, num_hidden)).astype(float)
  X_bar   = fun_sigmoid( np.dot(H_tilda , options['weights'].transpose()) + options['visible_biases'].repeat(batch_size, axis = 0) )
  H_bar   = fun_sigmoid( np.dot(X_bar   , options['weights'])             + options['hidden_biases' ].repeat(batch_size, axis = 0) )

  output  = {'H':H, 'H_tilda':H_tilda, 'X_bar':X_bar, 'H_bar': H_bar}

  # Learning
  options['delta_weights']        = options['mu'] * options['delta_weights'] + \
                                    options['learning_rate' ]                * \
                                    ( (np.dot(data.transpose() , H) - np.dot(X_bar.transpose() , H_bar) ) / batch_size - options['weight_cost'] * options['weights'] )       

  options['delta_hidden_biases']  = options['mu'] * options['delta_hidden_biases']  + \
                                    options['learning_rate' ]                       * (H.sum(axis = 0)    - H_bar.sum(axis = 0))

  options['delta_visible_biases'] = options['mu'] * options['delta_visible_biases'] + \
                                    options['learning_rate']                        * (data.sum(axis = 0) - X_bar.sum(axis = 0))

  options['weights']        = options['weights']        + options['delta_weights']
  options['hidden_biases']  = options['hidden_biases']  + options['delta_hidden_biases']
  options['visible_biases'] = options['visible_biases'] + options['delta_visible_biases']

  return options, output


In [ ]:
#@title 8.6.3. Data processing
# https://thispersondoesnotexist.com/ 
#!unzip gan_faces_32.zip
batch_files = glob.glob('/content/*.npy')

datain = np.empty((len(batch_files) * 200,32,32,1))
c = 0
for i0 in range(len(batch_files)):
  datain[c:c+200] = np.load(batch_files[i0])
  c             = c + 200

datain = datain/255

#flatten the data 
datain = datain.reshape(datain.shape[0], datain.shape[1] * datain.shape[2] * datain.shape[3])

# select an image
datain = datain[4:5,:]

In [ ]:
#@title 8.6.4. Training RBM
epochs     = 1000
num_hidden = 128

num_var    = datain.shape[1]
s_val      = int(np.sqrt(datain.shape[1]))

options = fun_rbm_options(num_hidden, num_var)

for i0 in range(epochs):

  if i0>2:
    options['mu']  = 0.5
  
  options, output = fun_rbm(datain, options)
  mse_val         = np.mean( np.sqrt( ((datain - output['X_bar'])**2).sum(axis = 1) ) )
  
  clear_output(wait=True)

  img_mat = np.concatenate((datain[:1,:].reshape(( int(s_val), int(s_val) )), output['X_bar'][:1,:].reshape(( int(s_val), int(s_val) ))), axis = 1)

  plt.imshow(img_mat, cmap='gray')
  plt.title('Epoch#: {:05.0f} | MSE: {:02.8f}'.format(i0+1, mse_val), fontsize = 15)
  plt.show()
